In [1]:
import numpy as np
import os
import glob
import random

In [123]:
data_folder = "languageID"
data_files = os.listdir(data_folder)

languages = ['e', 's', 'j']
training_files = []
for l in languages:
    for i in range(10):
        filename = f"{l}{i}.txt"
        file_path = os.path.join(data_folder, filename)
        if os.path.exists(file_path):
            training_files.append(filename)


file_language_counts = []
for l in languages:
    file_language_counts.append(len([file for file in training_files if file.startswith(l)]))

smooth_prior_probs = []
alpha = 0.5
for count in file_language_counts:
    smooth_prior_probs.append((count+alpha)/(sum(file_language_counts)+alpha*len(languages)))

In [124]:
print('p̂(y=e) = '+str(smooth_prior_probs[0]))
print('p̂(y=s) = '+str(smooth_prior_probs[1]))
print('p̂(y=j) = '+str(smooth_prior_probs[2]))

p̂(y=e) = 0.3333333333333333
p̂(y=s) = 0.3333333333333333
p̂(y=j) = 0.3333333333333333


In [126]:
def get_class_cond_prob(lang_files, language):
    char_counts = {char:0 for char in "abcdefghijklmnopqrstuvwxyz "}
    total_chars = 0
    for file in lang_files:
        with open(os.path.join("languageID/", file), "r") as f:
            all_text = f.read()
            for char in all_text:
                if char in char_counts:
                    char_counts[char] += 1
                    total_chars += 1
    
    char_cond_probs = []
    alpha = 0.5
    for char in "abcdefghijklmnopqrstuvwxyz ":
        N_c = char_counts[char]
        cond_prob = (N_c+alpha)/(total_chars+27*alpha)
        char_cond_probs.append(cond_prob)
        print(f"θ{char},{language} = {cond_prob:.5f}")
    return char_cond_probs

In [127]:
get_class_cond_prob([file for file in training_files if file.startswith('e')], 'e')

θa,e = 0.06017
θb,e = 0.01113
θc,e = 0.02151
θd,e = 0.02197
θe,e = 0.10537
θf,e = 0.01893
θg,e = 0.01748
θh,e = 0.04722
θi,e = 0.05541
θj,e = 0.00142
θk,e = 0.00373
θl,e = 0.02898
θm,e = 0.02052
θn,e = 0.05792
θo,e = 0.06446
θp,e = 0.01675
θq,e = 0.00056
θr,e = 0.05382
θs,e = 0.06618
θt,e = 0.08013
θu,e = 0.02666
θv,e = 0.00928
θw,e = 0.01550
θx,e = 0.00116
θy,e = 0.01384
θz,e = 0.00063
θ ,e = 0.17925


[0.0601685114819098,
 0.011134974392863043,
 0.021509995043779945,
 0.021972575582355856,
 0.1053692383941847,
 0.018932760614571286,
 0.017478936064761277,
 0.047216256401784236,
 0.055410540227986124,
 0.001420783082768875,
 0.0037336857756484387,
 0.028977366595076822,
 0.020518751032545846,
 0.057921691723112505,
 0.06446390219725756,
 0.01675202378985627,
 0.0005617049396993227,
 0.053824549810011564,
 0.06618205848339666,
 0.08012555757475633,
 0.026664463902197257,
 0.009284652238559392,
 0.015496448042293078,
 0.001156451346439782,
 0.013844374690236246,
 0.0006277878737815959,
 0.1792499586981662]

In [128]:
get_class_cond_prob([file for file in training_files if file.startswith('s')], 's')

θa,s = 0.10456
θb,s = 0.00823
θc,s = 0.03753
θd,s = 0.03975
θe,s = 0.11381
θf,s = 0.00860
θg,s = 0.00718
θh,s = 0.00453
θi,s = 0.04986
θj,s = 0.00663
θk,s = 0.00028
θl,s = 0.05294
θm,s = 0.02581
θn,s = 0.05418
θo,s = 0.07249
θp,s = 0.02427
θq,s = 0.00768
θr,s = 0.05930
θs,s = 0.06577
θt,s = 0.03561
θu,s = 0.03370
θv,s = 0.00589
θw,s = 0.00009
θx,s = 0.00250
θy,s = 0.00786
θz,s = 0.00268
θ ,s = 0.16826


[0.10456045141993771,
 0.008232863618143134,
 0.03752582405722919,
 0.039745922111559924,
 0.1138108599796491,
 0.00860287996053159,
 0.0071844839813758445,
 0.0045327001942585795,
 0.049859702136844375,
 0.006629459467793161,
 0.0002775122567913416,
 0.052943171656748174,
 0.02580863988159477,
 0.054176559464709693,
 0.07249236841293824,
 0.02426690512164287,
 0.007677839104560451,
 0.05929511886774999,
 0.06577040485954797,
 0.03561407295488884,
 0.03370232185254849,
 0.00588942678301625,
 9.250408559711388e-05,
 0.0024976103111220747,
 0.007862847275754679,
 0.0026826184823163022,
 0.16826493170115014]

In [129]:
get_class_cond_prob([file for file in training_files if file.startswith('j')], 'j')

θa,j = 0.13177
θb,j = 0.01087
θc,j = 0.00549
θd,j = 0.01723
θe,j = 0.06020
θf,j = 0.00388
θg,j = 0.01401
θh,j = 0.03176
θi,j = 0.09703
θj,j = 0.00234
θk,j = 0.05741
θl,j = 0.00143
θm,j = 0.03980
θn,j = 0.05671
θo,j = 0.09116
θp,j = 0.00087
θq,j = 0.00010
θr,j = 0.04280
θs,j = 0.04217
θt,j = 0.05699
θu,j = 0.07062
θv,j = 0.00024
θw,j = 0.01974
θx,j = 0.00003
θy,j = 0.01415
θz,j = 0.00772
θ ,j = 0.12345


[0.1317656102589189,
 0.010866906600510151,
 0.005485866033054963,
 0.01722631818022992,
 0.06020475907613823,
 0.003878542227191726,
 0.014011670568503443,
 0.03176211607673224,
 0.09703343932352633,
 0.0023411020650616725,
 0.05740941332681086,
 0.001432614696530277,
 0.03979873510604843,
 0.05671057688947902,
 0.09116321324993885,
 0.0008735455466648031,
 0.00010482546559977637,
 0.04280373178657535,
 0.0421747789929767,
 0.056990111464411755,
 0.07061742199238269,
 0.0002445927530661449,
 0.01974212935462455,
 3.4941821866592126e-05,
 0.01415143785596981,
 0.00772214263251686,
 0.12344945665466997]

In [131]:
def get_class_count(lang_files):
    char_counts = {char:0 for char in "abcdefghijklmnopqrstuvwxyz "}
    total_chars = 0
    for file in lang_files:
        with open(os.path.join("languageID/", file), "r") as f:
            all_text = f.read()
            for char in all_text:
                if char in char_counts:
                    char_counts[char] += 1
                    total_chars += 1
    char_counts_array = []
    for char in "abcdefghijklmnopqrstuvwxyz ":
        N_c = char_counts[char]
        char_counts_array.append(N_c)
        print(f"x_{char} = {N_c:.0f}")
    return char_counts_array

In [132]:
get_class_count([file for file in data_files if file.startswith('e10')])

x_a = 164
x_b = 32
x_c = 53
x_d = 57
x_e = 311
x_f = 55
x_g = 51
x_h = 140
x_i = 140
x_j = 3
x_k = 6
x_l = 85
x_m = 64
x_n = 139
x_o = 182
x_p = 53
x_q = 3
x_r = 141
x_s = 186
x_t = 225
x_u = 65
x_v = 31
x_w = 47
x_x = 4
x_y = 38
x_z = 2
x_  = 498


[164,
 32,
 53,
 57,
 311,
 55,
 51,
 140,
 140,
 3,
 6,
 85,
 64,
 139,
 182,
 53,
 3,
 141,
 186,
 225,
 65,
 31,
 47,
 4,
 38,
 2,
 498]

In [133]:
def get_log_likelihood(char_counts, class_cond_prob, vocab_size=27):
    net_likelihood = 0
    for i in range(vocab_size):
        net_likelihood += char_counts[i]*np.log10(class_cond_prob[i])
    return net_likelihood

In [134]:
all_log_likelihoods = []

for l in languages:
    all_log_likelihoods.append(get_log_likelihood(get_class_count([file for file in data_files if file.startswith('e10')]), get_class_cond_prob([file for file in training_files if file.startswith(l)], l)))

x_a = 164
x_b = 32
x_c = 53
x_d = 57
x_e = 311
x_f = 55
x_g = 51
x_h = 140
x_i = 140
x_j = 3
x_k = 6
x_l = 85
x_m = 64
x_n = 139
x_o = 182
x_p = 53
x_q = 3
x_r = 141
x_s = 186
x_t = 225
x_u = 65
x_v = 31
x_w = 47
x_x = 4
x_y = 38
x_z = 2
x_  = 498
θa,e = 0.06017
θb,e = 0.01113
θc,e = 0.02151
θd,e = 0.02197
θe,e = 0.10537
θf,e = 0.01893
θg,e = 0.01748
θh,e = 0.04722
θi,e = 0.05541
θj,e = 0.00142
θk,e = 0.00373
θl,e = 0.02898
θm,e = 0.02052
θn,e = 0.05792
θo,e = 0.06446
θp,e = 0.01675
θq,e = 0.00056
θr,e = 0.05382
θs,e = 0.06618
θt,e = 0.08013
θu,e = 0.02666
θv,e = 0.00928
θw,e = 0.01550
θx,e = 0.00116
θy,e = 0.01384
θz,e = 0.00063
θ ,e = 0.17925
x_a = 164
x_b = 32
x_c = 53
x_d = 57
x_e = 311
x_f = 55
x_g = 51
x_h = 140
x_i = 140
x_j = 3
x_k = 6
x_l = 85
x_m = 64
x_n = 139
x_o = 182
x_p = 53
x_q = 3
x_r = 141
x_s = 186
x_t = 225
x_u = 65
x_v = 31
x_w = 47
x_x = 4
x_y = 38
x_z = 2
x_  = 498
θa,s = 0.10456
θb,s = 0.00823
θc,s = 0.03753
θd,s = 0.03975
θe,s = 0.11381
θf,s = 0.00860
θg,s = 0.

In [135]:
all_log_likelihoods

[-3405.678891486211, -3677.2938684322726, -3809.3849846259345]

In [136]:
for i in range(3):
    print(f'log(p̂(x|y={languages[i]})) = {all_log_likelihoods[i]}')

log(p̂(x|y=e)) = -3405.678891486211
log(p̂(x|y=s)) = -3677.2938684322726
log(p̂(x|y=j)) = -3809.3849846259345


In [137]:
10**all_log_likelihoods[0]

0.0

In [138]:
!pip install mpmath



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [139]:
import mpmath
exp_values = []
for i in range(3):
    exp_values.append(mpmath.power(10, all_log_likelihoods[i]))

In [140]:
for i in range(3):
    print(f'p̂(x|y={languages[i]}) = {exp_values[i]}')

p̂(x|y=e) = 2.09463576084003e-3406
p̂(x|y=s) = 5.08313410694905e-3678
p̂(x|y=j) = 4.12112107646456e-3810


In [208]:
#net_count_vocab = get_class_count([file for file in training_files])

In [209]:
#net_count_vocab_prob = [i/sum(net_count_vocab) for i in net_count_vocab]

In [210]:
def get_posterior_denom(char_count, net_count_vocab_prob, vocab_size=27):
    constant_denom = 0
    for i in range(vocab_size):
        constant_denom += char_count[i]*np.log10(net_count_vocab_prob[i])
    return constant_denom

In [211]:
constant_denom = get_posterior_denom(get_class_count([file for file in data_files if file.startswith('e10')]), net_count_vocab_prob)

x_a = 164
x_b = 32
x_c = 53
x_d = 57
x_e = 311
x_f = 55
x_g = 51
x_h = 140
x_i = 140
x_j = 3
x_k = 6
x_l = 85
x_m = 64
x_n = 139
x_o = 182
x_p = 53
x_q = 3
x_r = 141
x_s = 186
x_t = 225
x_u = 65
x_v = 31
x_w = 47
x_x = 4
x_y = 38
x_z = 2
x_  = 498


In [212]:
all_posteriors = []
for i in range(3):
    post = np.log10(smooth_prior_probs[i])+all_log_likelihoods[i]  #-constant_denom
    all_posteriors.append(post)

In [213]:
for i in range(3):
    print(f'log(p̂(y={languages[i]}|x)) = {all_posteriors[i]}')

log(p̂(y=e|x)) = -3406.1560127409307
log(p̂(y=s|x)) = -3677.7709896869924
log(p̂(y=j|x)) = -3809.8621058806543


In [214]:
exp_posteriors = []
for i in range(3):
    exp_posteriors.append(mpmath.power(10, all_posteriors[i]))

In [215]:
for i in range(3):
    print(f'p̂(y={languages[i]}|x) = {exp_posteriors[i]}')

p̂(y=e|x) = 6.98211920279782e-3407
p̂(y=s|x) = 1.69437803564913e-3678
p̂(y=j|x) = 1.37370702548774e-3810


In [217]:
exp_posteriors[0]/sum(exp_posteriors), exp_posteriors[1]/sum(exp_posteriors), exp_posteriors[2]/sum(exp_posteriors)

(mpf('1.0'), mpf('2.4267389118337769e-272'), mpf('1.9674642978556946e-404'))

In [225]:
def get_prediction(file, languages, training_files, smooth_prior_probs):
    char_count_file = get_class_count([file])
    all_log_likelihoods = []
    for l in languages:
        all_log_likelihoods.append(get_log_likelihood(char_count_file, get_class_cond_prob([file for file in training_files if file.startswith(l)], l)))
    all_posteriors = []
    for i in range(3):
        post = np.log10(smooth_prior_probs[i])+all_log_likelihoods[i]
        all_posteriors.append(post)
    exp_posteriors = []
    for i in range(3):
        exp_posteriors.append(mpmath.power(10, all_posteriors[i]))
    return languages[np.argmax(np.array(exp_posteriors))]

In [226]:
test_files = [file for file in data_files if file not in training_files]

In [227]:
predictions_all_testfiles = []
for file in test_files:
    predictions_all_testfiles.append(get_prediction(file, languages, training_files, smooth_prior_probs))

x_a = 202
x_b = 30
x_c = 59
x_d = 86
x_e = 222
x_f = 10
x_g = 18
x_h = 13
x_i = 93
x_j = 13
x_k = 0
x_l = 85
x_m = 43
x_n = 112
x_o = 153
x_p = 42
x_q = 15
x_r = 120
x_s = 141
x_t = 78
x_u = 88
x_v = 5
x_w = 0
x_x = 4
x_y = 12
x_z = 7
x_  = 341
θa,e = 0.06017
θb,e = 0.01113
θc,e = 0.02151
θd,e = 0.02197
θe,e = 0.10537
θf,e = 0.01893
θg,e = 0.01748
θh,e = 0.04722
θi,e = 0.05541
θj,e = 0.00142
θk,e = 0.00373
θl,e = 0.02898
θm,e = 0.02052
θn,e = 0.05792
θo,e = 0.06446
θp,e = 0.01675
θq,e = 0.00056
θr,e = 0.05382
θs,e = 0.06618
θt,e = 0.08013
θu,e = 0.02666
θv,e = 0.00928
θw,e = 0.01550
θx,e = 0.00116
θy,e = 0.01384
θz,e = 0.00063
θ ,e = 0.17925
θa,s = 0.10456
θb,s = 0.00823
θc,s = 0.03753
θd,s = 0.03975
θe,s = 0.11381
θf,s = 0.00860
θg,s = 0.00718
θh,s = 0.00453
θi,s = 0.04986
θj,s = 0.00663
θk,s = 0.00028
θl,s = 0.05294
θm,s = 0.02581
θn,s = 0.05418
θo,s = 0.07249
θp,s = 0.02427
θq,s = 0.00768
θr,s = 0.05930
θs,s = 0.06577
θt,s = 0.03561
θu,s = 0.03370
θv,s = 0.00589
θw,s = 0.00009
θx,s 

In [228]:
groundtruth = [file[0] for file in test_files]

In [229]:
confusion_mat = {}
for l in languages:
    for k in languages:
        confusion_mat[(l, k)] = 0

In [230]:
for i in range(len(predictions_all_testfiles)):
    confusion_mat[predictions_all_testfiles[i], groundtruth[i]] += 1

In [231]:
from tabulate import tabulate

data = []
for l in languages:
    arr = []
    for k in languages:
        arr.append(confusion_mat[l, k])
    data.append(arr)
    
column_headers = ["", 'e', 'j', 'k']
row_headers = ['e', 'j', 'k']

table = tabulate(data, headers=column_headers, showindex=row_headers, tablefmt="grid")

In [232]:
print("\n")
print(table)



+----+-----+-----+-----+
|    |   e |   j |   k |
+====+=====+=====+=====+
| e  |  10 |   0 |   0 |
+----+-----+-----+-----+
| j  |   0 |  10 |   0 |
+----+-----+-----+-----+
| k  |   0 |   0 |  10 |
+----+-----+-----+-----+


In [233]:
input_file = 'languageID/e11.txt'
with open(input_file, 'r') as file:
    text = file.read()
    
all_chars = list(text)
random.shuffle(all_chars)

text_shuffled = ''.join(all_chars)
with open('languageID/shuffled_e11.txt', 'w') as file:
    file.write(text_shuffled)

In [234]:
get_prediction('shuffled_e11.txt', languages, training_files, smooth_prior_probs)

x_a = 199
x_b = 47
x_c = 70
x_d = 86
x_e = 352
x_f = 78
x_g = 47
x_h = 143
x_i = 170
x_j = 1
x_k = 15
x_l = 124
x_m = 59
x_n = 191
x_o = 236
x_p = 38
x_q = 3
x_r = 147
x_s = 194
x_t = 272
x_u = 86
x_v = 35
x_w = 57
x_x = 2
x_y = 43
x_z = 2
x_  = 618
θa,e = 0.06017
θb,e = 0.01113
θc,e = 0.02151
θd,e = 0.02197
θe,e = 0.10537
θf,e = 0.01893
θg,e = 0.01748
θh,e = 0.04722
θi,e = 0.05541
θj,e = 0.00142
θk,e = 0.00373
θl,e = 0.02898
θm,e = 0.02052
θn,e = 0.05792
θo,e = 0.06446
θp,e = 0.01675
θq,e = 0.00056
θr,e = 0.05382
θs,e = 0.06618
θt,e = 0.08013
θu,e = 0.02666
θv,e = 0.00928
θw,e = 0.01550
θx,e = 0.00116
θy,e = 0.01384
θz,e = 0.00063
θ ,e = 0.17925
θa,s = 0.10456
θb,s = 0.00823
θc,s = 0.03753
θd,s = 0.03975
θe,s = 0.11381
θf,s = 0.00860
θg,s = 0.00718
θh,s = 0.00453
θi,s = 0.04986
θj,s = 0.00663
θk,s = 0.00028
θl,s = 0.05294
θm,s = 0.02581
θn,s = 0.05418
θo,s = 0.07249
θp,s = 0.02427
θq,s = 0.00768
θr,s = 0.05930
θs,s = 0.06577
θt,s = 0.03561
θu,s = 0.03370
θv,s = 0.00589
θw,s = 0.00009


'e'